In [2]:
import pandas as pd
import sqlite3

In [3]:
#Step 1: Load CSV Data
orders = pd.read_csv(r"C:\Users\sathv\Downloads\orders.csv")
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [4]:
#Step 2: Load JSON Data
users = pd.read_json(r"C:\Users\sathv\Downloads\users.json")
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [5]:
#Step 3: Load SQL Data
db = sqlite3.connect("restaurants.db")
cursor = db.cursor()
with open(r"C:\Users\sathv\Downloads\restaurants.sql", "r", encoding="utf-8") as f:
    sql_script = f.read()
cursor.executescript(sql_script)
db.commit()

restaurants = pd.read_sql("SELECT * FROM restaurants", db)
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [6]:
orders_users = pd.merge(orders,users,on="user_id",how="left")
orders_users.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular


In [7]:
final_df = pd.merge(orders_users,restaurants,on="restaurant_id",how="left")
final_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [8]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)

In [9]:
high_total_rev = (final_df[final_df["membership"] == "Gold"].groupby("city")["total_amount"].sum().sort_values(ascending=False))
print(high_total_rev)

city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64


In [10]:
high_avg_value = (final_df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False))
print(high_avg_value)

cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64


In [11]:
user_total = (final_df.groupby("user_id")["total_amount"].sum())
(user_total > 1000).sum()

np.int64(2544)

In [25]:
ranges = {
    "3.0–3.5": final_df[(final_df["rating"] > 3.0) & (final_df["rating"] <= 3.5)],
    "3.6–4.0": final_df[(final_df["rating"] > 3.5) & (final_df["rating"] <= 4.0)],
    "4.1–4.5": final_df[(final_df["rating"] > 4.0) & (final_df["rating"] <= 4.5)],
    "4.6–5.0": final_df[(final_df["rating"] > 4.5) & (final_df["rating"] <= 5.0)]
}
high_rating_revenue = {
    key: frame["total_amount"].sum()
    for key, frame in ranges.items()
}
pd.Series(high_rating_revenue)

3.0–3.5    1881754.57
3.6–4.0    1717494.41
4.1–4.5    1960326.26
4.6–5.0    2197030.75
dtype: float64

In [26]:
high_avg_order = (final_df[final_df["membership"] == "Gold"].groupby("city")["total_amount"].mean().sort_values(ascending=False))
print(high_avg_order)

city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64


In [27]:
cuisine_low_res = (final_df.groupby("cuisine", as_index=False).agg(num_restaurants=("restaurant_id", "nunique"),revenue=("total_amount", "sum")).sort_values("num_restaurants"))
print(cuisine_low_res)

   cuisine  num_restaurants     revenue
0  Chinese              120  1930504.65
1   Indian              126  1971412.58
2  Italian              126  2024203.80
3  Mexican              128  2085503.09


In [28]:
gold_percentage = (final_df["membership"].eq("Gold").mean() * 100)
round(gold_percentage)

50

In [32]:

Res_high_avg = final_df.groupby("restaurant_name_x").agg(mean_value=("total_amount", "mean"), order_count=("order_id", "count")).reset_index()
Res_high_avg.query("order_count < 20") \
                .sort_values("mean_value", ascending=False)


,restaurant_name_x,mean_value,order_count
173,Hotel Dhaba Multicuisine,1040.222308,13
354,Sri Mess Punjabi,1029.180833,12
236,Ruchi Biryani Punjabi,1002.140625,16
343,Sri Delights Pure Veg,989.467222,18
88,Classic Kitchen Family Restaurant,973.167895,19
...,...,...,...
64,Annapurna Tiffins Punjabi,621.828947,19
126,Darbar Tiffins Non-Veg,596.815556,18
122,Darbar Restaurant Punjabi,589.972857,14
333,Spice Tiffins Pure Veg,578.578667,15


In [40]:
Combination_high_rev = final_df.groupby(["membership", "cuisine"])["total_amount"].sum().sort_values(ascending=False)
print(Combination_high_rev)

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64


In [42]:
(final_df["membership"] == "Gold").sum()

np.int64(4987)

In [43]:
round(final_df.loc[final_df["city"] == "Hyderabad", "total_amount"].sum())

1889367

In [44]:
final_df["user_id"].nunique()

2883

In [45]:
round(final_df.loc[final_df["membership"] == "Gold","total_amount"].mean())

797

In [46]:
(final_df["rating"] >= 4.5).sum()

np.int64(3374)